In [ ]:
ls

drive/  sample_data/


In [13]:
cd//content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [14]:
pwd

'/content/drive/MyDrive/Colab Notebooks'

In [ ]:
!unzip spam.csv

In [18]:
import pandas as pd
import numpy as np
import tensorflow
import seaborn as sns
import matplotlib.pyplot as plt
import keras
import re
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [19]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/spam.csv' ,delimiter=',', encoding="ISO-8859-1")

In [20]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [21]:
data.shape

(5572, 5)

In [22]:
data=data.drop(columns=["Unnamed: 2","Unnamed: 3","Unnamed: 4"])

In [23]:
data=data.rename({"v1":"Category","v2":"Message"},axis=1)

In [24]:
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [25]:
data["Message Length"]=data["Message"].apply(len)

In [26]:
ham_desc=data[data["Category"]=="ham"]["Message Length"].describe()
spam_desc=data[data["Category"]=="spam"]["Message Length"].describe()

In [27]:
data.describe(include="all")

,Category,Message,Message Length
count,5572,5572,5572.000000
unique,2,5169,NaN
top,ham,"Sorry, I'll call later",NaN
freq,4825,30,NaN
mean,NaN,NaN,80.118808
std,NaN,NaN,59.690841
min,NaN,NaN,2.000000
25%,NaN,NaN,36.000000
50%,NaN,NaN,61.000000
75%,NaN,NaN,121.000000


In [28]:
data["Category"].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [29]:
minority_len=len(data[data["Category"]=="spam"])
majority_len=len(data[data["Category"]=="ham"])

In [30]:
minority_indices=data[data["Category"]=="spam"].index
majority_indices=data[data["Category"]=="ham"].index

In [31]:
random_majority_indices=np.random.choice(majority_indices,size=minority_len,replace=False)

In [32]:
undersampled_indices=np.concatenate([minority_indices,random_majority_indices])

In [33]:
df=data.loc[undersampled_indices]

In [34]:
df=df.sample(frac=1)

In [35]:
df=df.reset_index()

In [36]:
df=df.drop(columns=["index"],)

In [37]:
df["Category"].value_counts()

ham     747
spam    747
Name: Category, dtype: int64

In [38]:
df["Label"]=df["Category"].map({"ham":0,"spam":1})

In [39]:
df.head()

,Category,Message,Message Length,Label
0,ham,Watching tv now. I got new job :),33,0
1,spam,U are subscribed to the best Mobile Content Se...,132,1
2,ham,Dear where you. Call me,23,0
3,spam,"You have won ?1,000 cash or a ?2,000 prize! To...",97,1
4,ham,Ok...,5,0


In [40]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
import nltk

In [42]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()

In [44]:
#declare empty list to store tokenized message
corpus=[]

#iterate through the df["Message"]
for message in df["Message"]:

    
    #replace every special characters, numbers etc.. with whitespace of message
    #It will help retain only letter/alphabets
  message=re.sub("[^a-zA-Z]"," ",message)
    
    #convert every letters to its lowercase
  message=message.lower()
  message=message.split()
    
    #perform stemming using PorterStemmer for all non-english-stopwords
  message=[stemmer.stem(words)
            for words in message
             if words not in set(stopwords.words("english"))
            ]
    #join the word lists with the whitespace
  message=" ".join(message)
    
    #append the message in corpus list
  corpus.append(message)

In [45]:
from tensorflow.keras.preprocessing.text import one_hot
vocab_size=10000
oneHot_doc=[one_hot(words,n=vocab_size)
           for words in corpus]

In [46]:
df["Message Length"].describe()

count    1494.000000
mean      106.198126
std        61.836297
min         3.000000
25%        50.000000
50%       120.000000
75%       153.000000
max       910.000000
Name: Message Length, dtype: float64

In [47]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sentence_len=200
embedded_doc=pad_sequences(oneHot_doc,maxlen=sentence_len,padding="pre")

In [48]:
extract_features=pd.DataFrame(data=embedded_doc)
target=df["Label"]

In [49]:
df_final=pd.concat([extract_features,target],axis=1)

In [50]:
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,Label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8432,6858,694,9811,5986,0
1,0,0,0,0,0,0,0,0,0,0,...,381,6294,9620,2522,3912,1494,4442,2624,1623,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2003,5550,0
3,0,0,0,0,0,0,0,0,0,0,...,9729,2657,5655,5550,3223,1599,3006,5920,3996,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3582,0


In [51]:
#split the dataframe into dependent and independent variables
X=df_final.drop("Label",axis=1)
y=df_final["Label"]

In [52]:
from sklearn.model_selection import train_test_split

In [53]:
X_trainval,X_test,y_trainval,y_test=train_test_split(X,y,random_state=42,test_size=0.15)

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X_trainval,y_trainval,random_state=42,test_size=0.15)

In [54]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential

In [55]:
model=Sequential()

In [56]:
feature_num=100
model.add(Embedding(input_dim=vocab_size,output_dim=feature_num,input_length=sentence_len))
model.add(LSTM(units=128))
model.add(Dense(units=1,activation="sigmoid"))

In [57]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.001),loss="binary_crossentropy",metrics=["accuracy"])


In [59]:
y_pred=model.predict(X_test)
y_pred=(y_pred>0.5)

8/8 [==============================] - 1s 96ms/step


In [60]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [61]:
score=accuracy_score(y_test,y_pred)
print("Test Score:{:.2f}%".format(score*100))

Test Score:64.44%


In [62]:
def classify_message(model,message):
    for sentences in message:
        sentences=nltk.sent_tokenize(message)
        for sentence in sentences:
            #replace all special characters
            words=re.sub("[^a-zA-Z]"," ",sentence)
            if words not in set(stopwords.words('english')):
                word=nltk.word_tokenize(words)
                word=" ".join(word)
        oneHot=[one_hot(word,n=vocab_size)]
    text=pad_sequences(oneHot,maxlen=sentence_len,padding="pre")
    predict=model.predict(text)
    
    #if predict value is greater than 0.5 its a spam
    if predict>0.5:
        print("It is a spam")
    #else the message is not a spam    
    else:
        print("It is not a spam")

In [63]:
message1="I am having a bad day and I would like to have a break today"
message2="This is to inform you had won a lottery and the subscription will end in a week so call us."

In [64]:
import nltk

In [65]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [66]:
classify_message(model,message1)

1/1 [==============================] - 0s 28ms/step
It is a spam


In [67]:
classify_message(model,message2)

1/1 [==============================] - 0s 29ms/step
It is not a spam
